In [1]:
import h5py
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import train_test_split
import pickle
import time

## Preprocess metadata

In [ ]:
# read metadata
path = 'your_path/fariness_data/HAM10000/'

demo_data = pd.read_csv(path + 'HAM10000_metadata.csv')
demo_data

In [ ]:
Counter(demo_data['dataset'])

In [22]:
# add image path to the metadata
pathlist = demo_data['image_id'].values.tolist()
paths = ['HAM10000_images/' + i + '.jpg' for i in pathlist]
demo_data['Path'] = paths

In [ ]:
# remove age/sex == null 
demo_data = demo_data[~demo_data['age'].isnull()]
demo_data = demo_data[~demo_data['sex'].isnull()]
demo_data

In [ ]:
# unify the value of sensitive attributes
sex = demo_data['sex'].values
sex[sex == 'male'] = 'M'
sex[sex == 'female'] = 'F'
demo_data['Sex'] = sex
demo_data

In [ ]:
# split subjects to different age groups
demo_data['Age_multi'] = demo_data['age'].values.astype('int')
demo_data['Age_multi'] = np.where(demo_data['Age_multi'].between(-1,19), 0, demo_data['Age_multi'])
demo_data['Age_multi'] = np.where(demo_data['Age_multi'].between(20,39), 1, demo_data['Age_multi'])
demo_data['Age_multi'] = np.where(demo_data['Age_multi'].between(40,59), 2, demo_data['Age_multi'])
demo_data['Age_multi'] = np.where(demo_data['Age_multi'].between(60,79), 3, demo_data['Age_multi'])
demo_data['Age_multi'] = np.where(demo_data['Age_multi']>=80, 4, demo_data['Age_multi'])

demo_data['Age_binary'] = demo_data['age'].values.astype('int')
demo_data['Age_binary'] = np.where(demo_data['Age_binary'].between(-1, 60), 0, demo_data['Age_binary'])
demo_data['Age_binary'] = np.where(demo_data['Age_binary']>= 60, 1, demo_data['Age_binary'])
demo_data

In [ ]:
# convert to binary labels
# benign: bcc, bkl, dermatofibroma, nv, vasc
# maglinant: akiec, mel

labels = demo_data['dx'].values.copy()
labels[labels == 'akiec'] = '1'
labels[labels == 'mel'] = '1'
labels[labels != '1'] = '0'

labels = labels.astype('int')

demo_data['binaryLabel'] = labels
demo_data

## Split train/val/test

In [27]:
def split_811(all_meta, patient_ids):
    sub_train, sub_val_test = train_test_split(patient_ids, test_size=0.2, random_state=0)
    sub_val, sub_test = train_test_split(sub_val_test, test_size=0.5, random_state=0)
    train_meta = all_meta[all_meta.lesion_id.isin(sub_train)]
    val_meta = all_meta[all_meta.lesion_id.isin(sub_val)]
    test_meta = all_meta[all_meta.lesion_id.isin(sub_test)]
    return train_meta, val_meta, test_meta

sub_train, sub_val, sub_test = split_811(demo_data, np.unique(demo_data['lesion_id']))

In [33]:
sub_train.to_csv('your_path/fariness_data/HAM10000/split/new_train.csv')
sub_val.to_csv('your_path/fariness_data/HAM10000/split/new_val.csv')
sub_test.to_csv('your_path/fariness_data/HAM10000/split/new_test.csv')

In [ ]:
# you can have a look of some examples here
img = cv2.imread('your_path/fariness_data/HAM10000/HAM10000_images/ISIC_0027419.jpg')
print(img.shape)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

## Save images into pickle files
This is optional, but if you are training many models, this step can save a lot of time by reducing the data IO.

In [ ]:
test_meta = pd.read_csv('your_path/fariness_data/HAM10000/split/new_train.csv')

path = 'your_path/fariness_data/HAM10000/pkls/'
images = []
start = time.time()
for i in range(len(test_meta)):

    img = cv2.imread(path + test_meta.iloc[i]['Path'])
    # resize to the input size in advance to save time during training
    img = cv2.resize(img, (256, 256))
    images.append(img)
    
end = time.time()
end-start
with open(path + 'train_images.pkl', 'wb') as f:
    pickle.dump(images, f)